<a href="https://colab.research.google.com/github/sallywang147/SCInvarinfer/blob/main/llama_for_smart_contracts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chatllama

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!git clone https://github.com/facebookresearch/llama.git

In [ ]:
!pip install -r /content/llama/requirements.txt

In [ ]:
!pip install -e /content/llama/.

In [ ]:
dir = '/content/drive/MyDrive/experiments/llama/LLaMA/7B' 
model_size = '7B'
path = '/tokenizer.model'

##Few-Shot Learning Test on Pre-trained LlaMA## 


In [ ]:
!torchrun --nproc_per_node 1 /content/llama/example.py --ckpt_dir "/content/drive/MyDrive/experiments/llama/LLaMA/7B/7B"  --tokenizer_path "/content/drive/MyDrive/experiments/llama/LLaMA/7B//tokenizer.model"

In [ ]:
!torchrun --nproc_per_node 1 /content/llama/example.py --ckpt_dir $dir/$model_size  --tokenizer_path $dir/$path

In [ ]:
!pip install chatllama-py

In [ ]:
!wget -O artifacts.zip https://nbllabartifacts.blob.core.windows.net/chatllama/artifacts.zip\?sp\=r\&st\=2023-03-08T14:53:24Z\&se\=2100-03-08T22:53:24Z\&spr\=https\&sv\=2021-06-08\&sr\=b\&sig\=jqr%2B2ZkR0SW9RjV0pDOdQ%2BDulLXLjbZ36vmNd4XxxyQ%3D
!unzip artifacts.zip

##preparing JSON files for fine-tuning##


In [ ]:
!pip install simplejson
from google.colab import auth
from google.auth import default
from collections import OrderedDict
import simplejson as json
from numpy import random
import pandas as pd
import gspread
import pandas
import xlrd
import json

#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
#defining my worksheet
worksheet = gc.open('invariants_line_number').sheet1
#get_all_values gives a list of rows
rows = worksheet.get_all_values()
#Convert to a DataFrame 
!mkdir datasets
cols = ['Source', 'Target']
df = pd.DataFrame(rows, columns=cols)
df = df.reset_index()  
df

In [ ]:
# List to hold dictionaries
def prepare_actor_model_data(df): 
    data_list = []
    for index, row in df.iterrows():
        data = OrderedDict()
        data["user_input"] = row['Source']
        data["completion"] = row['Target']
        data_list.append(data)
    # Serialize the list of dicts to JSON
    j = json.dumps(data_list)
    with open('datasets/actor_training_data.json', 'w') as f:
        f.write(j)

def prepare_rlhf_model_data(df): 
    data_list = []
    for index, row in df.iterrows():
        data = OrderedDict()
        data["user_input"] = row['Source']
        data_list.append(data)
    # Serialize the list of dicts to JSON
    j = json.dumps(data_list)
    with open('datasets/rlhf_training_data.json', 'w') as f:
        f.write(j)

def prepare_reward_model_data(df): 
    data_list = []
    for index, row in df.iterrows():
        data = OrderedDict()
        data["user_input"] = row['Source']
        data["completion"] = row['Target']
        data["score"] = 'None'
        data_list.append(data)
    # Serialize the list of dicts to JSON
    j = json.dumps(data_list)
    with open('datasets/reward_training_data.json', 'w') as f:
        f.write(j)

In [ ]:
prepare_actor_model_data(df)
prepare_rlhf_model_data(df)
prepare_reward_model_data(df)

In [ ]:
!python artifacts/main.py /content/artifacts/config/config.yaml --type ACTOR

In [ ]:
!cat /usr/local/lib/python3.9/dist-packages/chatllama/llama_model.py